In [1]:
import sys
sys.path.append('../Module')

from loader_csv import charger_csv
from inpute_na import traiter_valeurs_manquantes
from duplicata_csv import verifier_doublons
from display_value import display_unique_categorical_values
from force_data_format import force_data_format
from mode_utils import get_mode

# Spécifiez les chemins des quatre parties du CSV découpé
partie1 = "../Dataset/SBAnational_part_part1.csv"
partie2 = "../Dataset/SBAnational_part_part2.csv"
partie3 = "../Dataset/SBAnational_part_part3.csv"
partie4 = "../Dataset/SBAnational_part_part4.csv"

# Utilisez la fonction charger_csv pour obtenir un DataFrame avec toutes les données réassemblées
df = charger_csv(partie1, partie2, partie3, partie4)

/tmp/ipykernel_31072/1781239807.py:18: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  df = charger_csv(partie1, partie2, partie3, partie4)


In [2]:
df.shape

(899164, 27)

In [3]:
df.describe()

,LoanNr_ChkDgt,Zip,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural
count,8.991640e+05,899164.000000,899164.000000,899164.000000,899164.000000,899028.000000,899164.000000,899164.000000,899164.000000,899164.000000
mean,4.772612e+09,53804.391241,398660.950146,110.773078,11.411353,1.280404,8.430376,10.797257,2753.725933,0.757748
std,2.538175e+09,31184.159152,263318.312759,78.857305,74.108196,0.451750,236.688165,237.120600,12758.019136,0.646436
min,1.000014e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.589758e+09,27587.000000,235210.000000,60.000000,2.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,4.361439e+09,55410.000000,445310.000000,84.000000,4.000000,1.000000,0.000000,1.000000,1.000000,1.000000
75%,6.904627e+09,83704.000000,561730.000000,120.000000,10.000000,2.000000,1.000000,4.000000,1.000000,1.000000
max,9.996003e+09,99999.000000,928120.000000,569.000000,9999.000000,2.000000,8800.000000,9500.000000,99999.000000,2.000000


In [19]:
df[df['NAICS'] ==0]

,LoanNr_ChkDgt,City,State,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status
3,1000044001,BROKEN ARROW,OK,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,60,2,1.0,1,0,N,Y,P I F
4,1000054004,ORLANDO,FL,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,240,14,1.0,1,0,N,N,P I F
6,1000093009,UNION,NJ,WELLS FARGO BANK NATL ASSOC,SD,0,2-Jun-80,1980,45,45,2.0,0,0,N,N,CHGOFF
9,1000124001,GLASTONBURY,CT,WEBSTER BANK NATL ASSOC,CT,0,28-Feb-97,1997,84,3,2.0,1,0,N,Y,P I F
13,1000154010,SLATERSVILLE,RI,CITIZENS BANK NATL ASSOC,RI,0,28-Feb-97,1997,120,2,2.0,1,0,N,N,P I F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899153,9995413002,WOODBURY,NY,FLUSHING BANK,NY,0,27-Feb-97,1997,119,5,1.0,1,0,N,N,CHGOFF
899154,9995423005,CAMPBELL,OH,JPMORGAN CHASE BANK NATL ASSOC,IL,0,27-Feb-97,1997,60,1,1.0,1,0,N,N,P I F
899158,9995563001,IRVING,TX,LOANS FROM OLD CLOSED LENDERS,DC,0,27-Feb-97,1997,84,5,2.0,1,0,N,Y,P I F
899162,9995973006,HONOLULU,HI,BANK OF HAWAII,HI,0,27-Feb-97,1997,60,6,1.0,1,0,N,Y,CHGOFF


In [21]:
df.NAICS
NAICSTrue = 0
NAICSFalse = 0
for i in df.NAICS.astype(str):
    if len(i) == 6:
        NAICSTrue += 1
    else:
        NAICSFalse += 1
print(NAICSTrue)
print(NAICSFalse)

694223
198808


In [ ]:
df = df[df.NAICS != 0]


In [4]:
# Filter rows based on conditions
missing_mis_status = df[(df['MIS_Status'].isna() & df['ChgOffDate'].notna()) | df['MIS_Status'].isna() & (df['ChgOffPrinGr'] != 0.0)]

# Print relevant columns
print(missing_mis_status[['MIS_Status', 'ChgOffDate', 'ChgOffPrinGr']])

# Update 'MIS_Status' for filtered rows
df.loc[missing_mis_status.index, 'MIS_Status'] = 'CHGOFF'

       MIS_Status ChgOffDate  ChgOffPrinGr
343           NaN        NaN        $0.00 
611           NaN  22-May-90  $144,461.00 
738           NaN        NaN        $0.00 
740           NaN   8-Feb-90  $142,666.00 
833           NaN        NaN        $0.00 
...           ...        ...           ...
892340        NaN        NaN        $0.00 
893791        NaN        NaN        $0.00 
894290        NaN        NaN        $0.00 
896318        NaN        NaN        $0.00 
898182        NaN        NaN        $0.00 

[1997 rows x 3 columns]


In [5]:
print(df.loc[missing_mis_status.index, 'MIS_Status'])

343       CHGOFF
611       CHGOFF
738       CHGOFF
740       CHGOFF
833       CHGOFF
           ...  
892340    CHGOFF
893791    CHGOFF
894290    CHGOFF
896318    CHGOFF
898182    CHGOFF
Name: MIS_Status, Length: 1997, dtype: object


In [6]:
missing_bank_s = df[(df['BankState'].isna() & df['Bank'].isna())]

print(missing_bank_s.columns)
print(missing_bank_s[['BankState', 'Bank', 'City', 'State', 'MIS_Status']])

Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv'],
      dtype='object')
       BankState Bank        City State MIS_Status
407          NaN  NaN  MOUNTVILLE    PA     CHGOFF
611          NaN  NaN   DANIELSON    CT     CHGOFF
680          NaN  NaN   CLEVELAND    OH     CHGOFF
740          NaN  NaN  SIOUX CITY    IA     CHGOFF
6569         NaN  NaN    KENTWOOD    MI     CHGOFF
...          ...  ...         ...   ...        ...
888537       NaN  NaN     SEATTLE    WA     CHGOFF
888681       NaN  NaN     SEATTLE    WA     CHGOFF
897114       NaN  NaN  PORTSMOUTH    IA      P I F
897474       NaN  NaN  FORT WAYNE    IN      P I F
897749       NaN  NaN      DALLAS    T

In [7]:
# Compter les occurrences de chaque catégorie dans la colonne 'MIS_Status'
mis_status_counts = missing_bank_s['MIS_Status'].value_counts()
print(mis_status_counts)


P I F     1436
CHGOFF     123
Name: MIS_Status, dtype: int64


In [8]:
df = df.drop(missing_bank_s.index, axis=0)
df.shape

(897605, 27)

In [11]:
# Remplacer les valeurs manquantes dans 'City' par la valeur la plus fréquente pour cet État
df['City'] = df.groupby('State')['City'].transform(lambda x: x.fillna(x.mode()[0]))

# Remplacer les valeurs manquantes dans 'State' par la valeur la plus fréquente pour cette ville
df['State'] = df.groupby('City')['State'].transform(lambda x: x.fillna(x.mode()[0]))

# Supprimer les lignes avec des valeurs manquantes dans 'City' et 'State'
df.dropna(subset=['City', 'State'], inplace=True)

# Afficher les dimensions du DataFrame après le traitement
print("Nombre de lignes après le traitement des valeurs manquantes dans 'City' et 'State':", len(df))

Nombre de lignes après le traitement des valeurs manquantes dans 'City' et 'State': 897592


In [12]:
# Sélectionner les observations avec BankState manquant et Bank non manquant
missing_Bank_State = df[(df['BankState'].isna() & df['Bank'].notna())]

# Afficher les valeurs de BankState et Bank pour les observations concernées
print(missing_Bank_State[['BankState', 'Bank']])

# Modifier les valeurs de la colonne BankState à 'HS'
df.loc[missing_Bank_State.index, 'BankState'] = 'HS'

       BankState                            Bank
145106       NaN             KOREA EXCHANGE BANK
424311       NaN  STANDARD CHARTERED BK KOREA LT
444679       NaN    HONGKONG & SHANGHAI BK. CORP
447971       NaN  STANDARD CHARTERED BK KOREA LT
674674       NaN    HONGKONG & SHANGHAI BK. CORP
793950       NaN  THE BANK OF EAST ASIA, LIMITED
859233       NaN  THE BANK OF EAST ASIA, LIMITED


In [13]:
# Afficher les valeurs de BankState et Bank pour les observations concernées après la modification
print(df.loc[missing_Bank_State.index, ['BankState', 'Bank']])

       BankState                            Bank
145106        HS             KOREA EXCHANGE BANK
424311        HS  STANDARD CHARTERED BK KOREA LT
444679        HS    HONGKONG & SHANGHAI BK. CORP
447971        HS  STANDARD CHARTERED BK KOREA LT
674674        HS    HONGKONG & SHANGHAI BK. CORP
793950        HS  THE BANK OF EAST ASIA, LIMITED
859233        HS  THE BANK OF EAST ASIA, LIMITED


In [16]:
df['RevLineCr'].value_counts()

N    418755
0    257584
Y    201385
T     15284
1        23
R        14
`        11
2         6
C         2
3         1
,         1
7         1
A         1
5         1
.         1
4         1
-         1
Q         1
Name: RevLineCr, dtype: int64

In [17]:
mapping = {'0': 'N', '1': 'Y'}

# Remplacer les valeurs selon le mapping défini
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

# Supprimer les valeurs qui ne sont pas 'N', 'Y', ou 'T'
valid_values = ['N', 'Y', 'T']
df = df[df['RevLineCr'].isin(valid_values)]

# Afficher les nouvelles valeurs uniques pour vérification
print(df['RevLineCr'].value_counts())

N    676339
Y    201408
T     15284
Name: RevLineCr, dtype: int64


In [14]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
maybe_leaking_feature = ['SBA_Appv', 'GrAppv', 'RetainedJob', 'CreateJob']
useless_feature = ['Name', 'Zip']
target = ['MIS_status']
df = df.drop(leaking_feature, axis=1)
df = df.drop(maybe_leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.shape

(897592, 16)

In [ ]:
duplicates = df[df.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [LoanNr_ChkDgt, City, State, Bank, BankState, NAICS, Term, NoEmp, NewExist, CreateJob, FranchiseCode, UrbanRural, LowDoc, MIS_Status]
Index: []


In [ ]:
# Utiliser la fonction pour afficher les valeurs uniques de chaque colonne
display_unique_categorical_values(df)

Colonne 'City':
- LOS ANGELES: 11529 (1.28%)
- HOUSTON: 10226 (1.14%)
- NEW YORK: 7834 (0.87%)
- CHICAGO: 6019 (0.67%)
- MIAMI: 5591 (0.62%)
- SAN DIEGO: 5353 (0.60%)
- DALLAS: 5053 (0.56%)
- PHOENIX: 4492 (0.50%)
- LAS VEGAS: 4372 (0.49%)
- SPRINGFIELD: 3736 (0.42%)
- BROOKLYN: 3726 (0.42%)
- DENVER: 3544 (0.39%)
- SAN ANTONIO: 3514 (0.39%)
- SALT LAKE CITY: 3509 (0.39%)
- AUSTIN: 3480 (0.39%)
- SEATTLE: 3463 (0.39%)
- SAN FRANCISCO: 3360 (0.37%)
- PORTLAND: 3178 (0.35%)
- COLUMBUS: 3176 (0.35%)
- PHILADELPHIA: 3166 (0.35%)
- ATLANTA: 3116 (0.35%)
- ROCHESTER: 2911 (0.32%)
- INDIANAPOLIS: 2865 (0.32%)
- EL PASO: 2729 (0.30%)
- SAN JOSE: 2600 (0.29%)
- PITTSBURGH: 2553 (0.28%)
- SACRAMENTO: 2529 (0.28%)
- MINNEAPOLIS: 2390 (0.27%)
- LOUISVILLE: 2359 (0.26%)
- JACKSONVILLE: 2194 (0.24%)
- ORLANDO: 2154 (0.24%)
- KANSAS CITY: 2144 (0.24%)
- FORT WORTH: 2134 (0.24%)
- ARLINGTON: 2083 (0.23%)
- CHARLOTTE: 2071 (0.23%)
- WASHINGTON: 2055 (0.23%)
- OMAHA: 2042 (0.23%)
- CINCINNATI: 1979 (0.2

In [18]:
df.isna().sum()

LoanNr_ChkDgt       0
City                0
State               0
Bank                0
BankState           0
NAICS               0
ApprovalDate        0
ApprovalFY          0
Term                0
NoEmp               0
NewExist          136
FranchiseCode       0
UrbanRural          0
RevLineCr           0
LowDoc           2582
MIS_Status          0
dtype: int64

In [ ]:
df.isna().sum()/df.count()

LoanNr_ChkDgt    0.000000
City             0.000031
State            0.000014
Bank             0.000000
BankState        0.000000
NAICS            0.000000
Term             0.000000
NoEmp            0.000000
NewExist         0.000152
CreateJob        0.000000
FranchiseCode    0.000000
UrbanRural       0.000000
LowDoc           0.002885
MIS_Status       0.000000
dtype: float64

In [ ]:
df.head()

,LoanNr_ChkDgt,City,State,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,FranchiseCode,UrbanRural,LowDoc,MIS_Status
0,1000014003,EVANSVILLE,IN,FIFTH THIRD BANK,OH,451120,84,4,2.0,0,1,0,Y,P I F
1,1000024006,NEW PARIS,IN,1ST SOURCE BANK,IN,722410,60,2,2.0,0,1,0,Y,P I F
2,1000034009,BLOOMINGTON,IN,GRANT COUNTY STATE BANK,IN,621210,180,7,1.0,0,1,0,N,P I F
3,1000044001,BROKEN ARROW,OK,1ST NATL BK & TR CO OF BROKEN,OK,0,60,2,1.0,0,1,0,Y,P I F
4,1000054004,ORLANDO,FL,FLORIDA BUS. DEVEL CORP,FL,0,240,14,1.0,7,1,0,N,P I F


In [ ]:
# Supposons que df est votre DataFrame
doublons, df_sans_doublons = verifier_doublons(df)
print(doublons)

In [ ]:
df.duplicated().sum()

0

In [ ]:
print(df.columns)


Index(['LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'NAICS', 'Term',
       'NoEmp', 'NewExist', 'CreateJob', 'FranchiseCode', 'UrbanRural',
       'LowDoc', 'MIS_Status'],
      dtype='object')


In [ ]:
dogshit_leaking = ['MIS_Status', 'ChgOffPrinGr']
df = df.drop(dogshit_leaking, axis=1)
df.shape

KeyError: "['ChgOffPrinGr'] not found in axis"

In [ ]:
# Exclure les colonnes non numériques
df_numeric = df.select_dtypes(include=['int64', 'float64'])

# Calculer la matrice de corrélation
correlation_matrix = df_numeric.corr()

# Afficher la matrice de corrélation
print(correlation_matrix)